In [2]:
# !pip install info-nce-pytorch

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.auto import tqdm
# from info_nce import InfoNCE, info_nce

class TextImageEmbeddingDataset(Dataset):
    def __init__(self, text_embeddings, image_embeddings):
        """
        Args:
            text_embeddings: torch.Tensor (N, 1024)
            image_embeddings: torch.Tensor (N, 1536)
        """
        self.text_embeddings = text_embeddings
        self.image_embeddings = image_embeddings

    def __len__(self):
        return len(self.text_embeddings)

    def __getitem__(self, idx):
        text_emb = self.text_embeddings[idx]
        img_emb = self.image_embeddings[idx]
        return text_emb, img_emb


def create_dataloader(text_tensor, image_tensor, batch_size=64, shuffle=True, num_workers=0):
    dataset = TextImageEmbeddingDataset(text_tensor, image_tensor)
    loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        num_workers=num_workers
    )
    return loader

In [3]:
print("=== DATA LOADING & SETUP ===")

BATCH_SIZE = 256
VAL_SPLIT = 0.1
RANDOM_STATE = 11

data = np.load("/kaggle/input/aml-competition/train/train/train.npz")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

# Apply normalization on embeddings before training.
text_embeddings_cpu = F.normalize(torch.tensor(data["captions/embeddings"], dtype=torch.float32), dim=1)
image_embeddings_small_cpu = F.normalize(torch.tensor(data["images/embeddings"], dtype=torch.float32), dim=1)
image_embeddings_cpu = image_embeddings_small_cpu.repeat_interleave(5, dim=0)

# Create train/validation split
indices = np.arange(len(text_embeddings_cpu))
train_indices, val_indices = train_test_split(
    indices, 
    test_size=VAL_SPLIT, 
    random_state=RANDOM_STATE
)

text_train = text_embeddings_cpu[train_indices]
img_train = image_embeddings_cpu[train_indices]

text_val = text_embeddings_cpu[val_indices]
img_val = image_embeddings_cpu[val_indices]

train_loader = create_dataloader(text_train, img_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = create_dataloader(text_val, img_val, batch_size=BATCH_SIZE*2, shuffle=False)

print(f"Training samples: {len(text_train)}, Validation samples: {len(text_val)}")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB)")
else:
    print("CUDA not available. Training will be slow.")

=== DATA LOADING & SETUP ===
Training samples: 112500, Validation samples: 12500
Using device: cuda
GPU: Tesla T4 (15.8 GB)


In [99]:
class ResidualMLP(nn.Module):
    def __init__(self, input_dim=1024, hidden_dim=2048, output_dim=1536, dropout=0.2, use_ln=True):
        super().__init__()
                
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.norm1 = nn.LayerNorm(hidden_dim)
        
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.norm2 = nn.LayerNorm(output_dim)
        
        self.act = nn.GELU()
        self.dropout = nn.Dropout(dropout)
        
        # Residual projection (only if dimensions differ)
        self.projection = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        identity = self.projection(x)

        out = self.fc1(x)
        out = self.norm1(out)
        out = self.act(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.norm2(out)
        
        out = out + identity
        out = self.act(out)
        return out

class InfoNCELoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, pred_embeds, target_embeds):
        # Normalize embeddings
        pred_embeds = F.normalize(pred_embeds, dim=1)
        target_embeds = F.normalize(target_embeds, dim=1)

        # Calculate logits (N, N) matrix
        # (pred @ target.T) is the matrix of all-to-all cosine similarities
        logits = (pred_embeds @ target_embeds.T) / self.temperature
        
        # Labels: diagonal elements (i, i) are the positive pairs
        batch_size = pred_embeds.shape[0]
        labels = torch.arange(batch_size, device=pred_embeds.device)
        
        return F.cross_entropy(logits, labels)

mlp = ResidualMLP().to(device)
loss_function = InfoNCELoss().to(device)
# loss_function = InfoNCE().to(device)
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)
# Mixed Precision for speeding up GPU training
scaler = torch.amp.GradScaler(device_str)

In [100]:
num_epochs = 100
patience = 3
best_val_loss = float('inf')
patience_counter = 0

num_train_steps = num_epochs * len(train_loader)
scheduler = CosineAnnealingLR(optimizer, T_max=num_train_steps)

print(f"Starting training for max {num_epochs} epochs...")
print(f"Batch size: {BATCH_SIZE}. Optimizer: Adam. Loss: InfoNCE.")
print(f"Features: Mixed Precision, Cosine LR Scheduler, Early Stopping (patience={patience}).")

for epoch in range(num_epochs):
    # --- Training Phase ---
    mlp.train()
    epoch_train_losses = []
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
    
    for text_batch, image_batch in train_loop:
        text_batch = text_batch.to(device)
        image_batch = image_batch.to(device)
        
        optimizer.zero_grad()
        
        with torch.amp.autocast(device_str, dtype=torch.float16):
            output = mlp(text_batch)
            loss = loss_function(output, image_batch)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        scheduler.step()
        
        epoch_train_losses.append(loss.item())
        train_loop.set_postfix(loss=np.mean(epoch_train_losses[-20:])) # Moving avg loss
        
    avg_train_loss = np.mean(epoch_train_losses)

    # --- Validation Phase ---
    mlp.eval()
    epoch_val_losses = []
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)
    
    with torch.no_grad():
        for text_batch, image_batch in val_loop:
            text_batch = text_batch.to(device)
            image_batch = image_batch.to(device)
            
            with torch.amp.autocast(device_str, dtype=torch.float16):
                output = mlp(text_batch)
                loss = loss_function(output, image_batch)
                
            epoch_val_losses.append(loss.item())
            val_loop.set_postfix(val_loss=np.mean(epoch_val_losses))

    avg_val_loss = np.mean(epoch_val_losses)
    
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.6f} - Val Loss: {avg_val_loss:.6f}")

    # --- Early Stopping & Model Checkpointing ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(mlp.state_dict(), 'pipeline_best_model.pth')
        print(f"✅ New best model saved! Val Loss: {best_val_loss:.6f}")
    else:
        patience_counter += 1
        print(f"⚠️ No improvement. Patience: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print(f"🛑 Early stopping triggered after {epoch+1} epochs.")
        break

print("\n Training complete.")
print(f"Loading best model from checkpoint (Val Loss: {best_val_loss:.6f})...")

# Load the best model state for inference
mlp.load_state_dict(torch.load('pipeline_best_model.pth'))
print("Best model loaded.")

Starting training for max 100 epochs...
Batch size: 256. Optimizer: AdamW. Loss: InfoNCE.
Features: Mixed Precision, Cosine LR Scheduler, Early Stopping (patience=3).


Epoch 1/100 [Train]:   0%|          | 0/440 [00:00<?, ?it/s]

Epoch 1/100 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1/100 - Train Loss: 3.461193 - Val Loss: 3.638513
✅ New best model saved! Val Loss: 3.638513


Epoch 2/100 [Train]:   0%|          | 0/440 [00:00<?, ?it/s]

...

Epoch 47/100 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 47/100 - Train Loss: 1.548444 - Val Loss: 2.606104
⚠️ No improvement. Patience: 2/3


Epoch 48/100 [Train]:   0%|          | 0/440 [00:00<?, ?it/s]

Epoch 48/100 [Val]:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch 48/100 - Train Loss: 1.545602 - Val Loss: 2.606025
⚠️ No improvement. Patience: 3/3
🛑 Early stopping triggered after 48 epochs.

✅ Training complete.
Loading best model from checkpoint (Val Loss: 2.603633)...
Best model loaded.


In [101]:
# Evaluate model on training data

mlp.eval()
eval_losses = []
with torch.no_grad():
    for i, (text_batch, image_batch) in enumerate(train_loader):
        if i >= 5:
            break
        text_batch = text_batch.to(device)
        image_batch = image_batch.to(device)
        output = mlp(text_batch)
        loss = loss_function(output, image_batch)
        eval_losses.append(loss.item())

print(f"Training Eval Avg Loss: {np.mean(eval_losses):.6f}")

Training Eval Avg Loss: 1.322532


In [102]:
# Load test data

test_data = np.load("/kaggle/input/aml-competition/test/test/test.clean.npz")

caption_ids = test_data["captions/ids"]
caption_texts = test_data["captions/text"]

test_text_embeddings_raw = torch.tensor(test_data["captions/embeddings"], dtype=torch.float32)

# Normalize test embeddings
test_text_embeddings = F.normalize(test_text_embeddings_raw, dim=1).to(device)

print(f"Loaded and normalized {len(caption_ids)} test captions.")


=== TESTING PHASE ===
Loaded and normalized 1500 test captions.


In [103]:
# Generate predictions

predictions = []
batch_size = 256
mlp.eval()

print("Generating predictions...")
with torch.no_grad():
    for i in tqdm(range(0, len(test_text_embeddings), batch_size), desc="Predicting"):
        batch = test_text_embeddings[i:i+batch_size]
        
        with torch.amp.autocast(device_str, dtype=torch.float16):
            predicted_batch = mlp(batch)
            
        predictions.append(predicted_batch.float().cpu().numpy())

predictions = np.concatenate(predictions, axis=0)
print(f"Generated embeddings for {len(predictions)} captions.")

Generating predictions...


Predicting:   0%|          | 0/6 [00:00<?, ?it/s]

Generated embeddings for 1500 captions.


In [104]:
# Evaluate cosine similarity on subset

n_eval = min(500, len(text_embeddings_cpu))

# Get subsets and move to device
eval_text_subset = text_embeddings_cpu[:n_eval].to(device)
eval_img_subset = image_embeddings_cpu[:n_eval].to(device)

mlp.eval()
with torch.no_grad():
    with torch.amp.autocast(device_str, dtype=torch.float16):
        eval_predicted = mlp(eval_text_subset)

    # F.cosine_similarity handles mixed precision (float16 preds, float32 targets)
    cosine_sim_gpu = F.cosine_similarity(eval_predicted, eval_img_subset, dim=1)
    cosine_scores = cosine_sim_gpu.cpu().numpy()

print(f"Average cosine similarity: {np.mean(cosine_scores):.4f}")
print(f"Std: {np.std(cosine_scores):.4f}")
print(f"Range: [{np.min(cosine_scores):.4f}, {np.max(cosine_scores):.4f}]")


Running sanity check (cosine similarity on train subset)...
Average cosine similarity: 0.3817
Std: 0.0796
Range: [0.0455, 0.5437]


In [105]:
# Create submission CSV

submission_data = [
    {'id': caption_id, 'embedding': predictions[i].tolist()}
    for i, caption_id in enumerate(caption_ids)
]

submission_df = pd.DataFrame(submission_data)
submission_df.to_csv('submission23.csv', index=False, quoting=0)

print(f"\n✅ Submission saved with {len(submission_df)} rows.")
submission_df.head(2)


✅ Submission saved with 1500 rows.


,id,embedding
0,1,"[-0.0206146240234375, 0.018218994140625, 0.017..."
1,2,"[-0.033355712890625, -0.10308837890625, -0.008..."
